In [10]:
import weaviate
import os

cohere_key = os.environ["COHERE_API_KEY"]

client = weaviate.connect_to_local(
    headers={"X-Cohere-Api-Key": cohere_key}
)

In [11]:
if client.collections.exists("Wiki"):
    client.collections.delete("Wiki")

In [12]:
from weaviate.classes.config import Configure, Property, DataType, Tokenization

wiki = client.collections.create(
    name="Wiki",
    properties=[
        Property(
            name="title",
            data_type=DataType.TEXT,
            tokenization=Tokenization.KAGOME_KR
        ),
        Property(
            name="chunk_default_tokenizer",
            data_type=DataType.TEXT,
        ),
        Property(
            name="chunk",
            data_type=DataType.TEXT,
            tokenization=Tokenization.KAGOME_KR
        ),
    ],
    vectorizer_config=[
        Configure.NamedVectors.text2vec_cohere(
            name="title",
            source_properties=["title"],
            model="embed-multilingual-v3.0"
        ),
        Configure.NamedVectors.text2vec_cohere(
            name="chunk",
            source_properties=["chunk"],
            model="embed-multilingual-v3.0"
        ),
    ],
    generative_config=Configure.Generative.cohere(model="command-r-plus")
)

In [13]:
from pathlib import Path

data_dir = Path("./data")
src_texts = [
    {"body": txt_file.read_text(), "title": txt_file.stem}
    for txt_file in data_dir.glob("*.txt")
]

In [14]:
from typing import List

def get_chunks_marker(text: str, marker: str = "\n\n") -> List[str]:
    return text.split(marker)

def get_chunks_fixed_size(text: str, chunk_size: int) -> List[str]:
    overlap = int(chunk_size // 4)
    return [text[i:i+chunk_size+overlap] for i in range(0, len(text), chunk_size)]

def get_chunks(text: str) -> List[str]:
    sections = get_chunks_marker(text)
    chunks = []
    for s in sections:
        if len(s) > 100:
            sub_chunks = get_chunks_fixed_size(s, 50)
            chunks.extend(sub_chunks)
        else:
            chunks.append(s)
    return chunks

In [15]:
from weaviate.util import generate_uuid5

with wiki.batch.fixed_size(batch_size=200) as batch:
    for src_text in src_texts:
        chunks = get_chunks(src_text["body"])
        for chunk in chunks:
            batch.add_object(
                properties={
                    "title": src_text["title"],
                    "chunk": chunk,
                },
                uuid=generate_uuid5(chunk)
            )

In [16]:
wiki.aggregate.over_all(total_count=True)

AggregateReturn(properties={}, total_count=416)

In [17]:
r = wiki.generate.bm25(
    query="머리",
    query_properties=["chunk"],
    single_prompt="Return a translation of this into English (and nothing else): {chunk}",
    limit=3
)
for i, o in enumerate(r.objects):
    print(f"\n========== RESULT {i+1} ==========")
    print("ARTICLE TITLE:", o.properties["title"])
    print("CHUNK BODY:", o.properties["chunk"].replace("\n", " ")[:100] + "...")
    print("TRANSLATION:", o.generated.replace("\n", " ")[:100] + "...")


========== RESULT 1 ==========
ARTICLE TITLE: preface
CHUNK BODY: 적으로 머리말을 만들고 유지하는 기능을 제공하며 여기서 머리말은 페이지마다 동일할 수도 있고 페이지 번호와 같이...
TRANSLATION: It provides the ability to create and maintain headers as an enemy, where the header can be the same...

========== RESULT 2 ==========
ARTICLE TITLE: preface
CHUNK BODY: '''머리말''' 또는 '''머리글'''은 [[타이포그래피]]에서 본문과 구별되면서도 인쇄된 페이지의 꼭대기에 ...
TRANSLATION: A '''preface''' or '''header''' is a distinct piece of text from the main body, appearing at the top...

========== RESULT 3 ==========
ARTICLE TITLE: preface
CHUNK BODY: [[출판]]에서 머리말은 난외표제(running head)로도 불린다. 책에서의 일반적인 난외표제는 펼친 책의 ...
TRANSLATION: In publishing, a frontispiece is also called a running head. A common running head in a book is a ti...


In [9]:
r = wiki.generate.bm25(
    query="머리말",
    query_properties=["chunk"],
    single_prompt="Return a translation of this into English (and nothing else): {chunk}",
    limit=3
)
for i, o in enumerate(r.objects):
    print(f"\n========== RESULT {i+1} ==========")
    print("ARTICLE TITLE:", o.properties["title"])
    print("CHUNK BODY:", o.properties["chunk"].replace("\n", " ")[:100] + "...")
    print("TRANSLATION:", o.generated.replace("\n", " ")[:100] + "...")


========== RESULT 1 ==========
ARTICLE TITLE: head
CHUNK BODY: 아주 단순한 동물의 경우 머리가 없는 것도 있으나 대부분의 [[좌우 대칭 동물류]]는 머리가 있다. [[척추동물...
TRANSLATION: Even among very simple animals, some have no head, but most [[bilaterian]] animals have a head. [[ve...

========== RESULT 2 ==========
ARTICLE TITLE: head
CHUNK BODY:  머리 그림]] '''머리'''({{llang|en|Head}})는 [[인간]]이나 [[동물]]의 [[목]] 위...
TRANSLATION: 'Head' (in English) is the part of the body that is above the neck in humans and animals....

========== RESULT 3 ==========
ARTICLE TITLE: head
CHUNK BODY:  있다. 동물의 머리는 '''대가리'''라고 부른다....
TRANSLATION: There is a separate word for the head of an animal in Korean. The word is 'daegari'....


In [12]:
r = wiki.generate.bm25(
    query="머리구성",
    query_properties=["chunk"],
    single_prompt="Return a translation of this into English (and nothing else): {chunk}",
    limit=3
)
for i, o in enumerate(r.objects):
    print(f"\n========== RESULT {i+1} ==========")
    print("ARTICLE TITLE:", o.properties["title"])
    print("CHUNK BODY:", o.properties["chunk"].replace("\n", " ")[:100] + "...")
    print("TRANSLATION:", o.generated.replace("\n", " ")[:100] + "...")


========== RESULT 1 ==========
ARTICLE TITLE: skull
CHUNK BODY: == 구조 == [[파일:Lateral head skull.jpg|섬네일|왼쪽|머리의 구성]] 머리뼈는 얼굴을 ...
TRANSLATION: == Structure == [[File:Lateral head skull.jpg|thumb|left|Composition of the head]] The skull is comp...

========== RESULT 2 ==========
ARTICLE TITLE: skull
CHUNK BODY: ]] 머리뼈는 얼굴을 구성하고 머리뼈공간을 보호한다. [[뇌]]를 비롯하여 [[눈 (해부학)|눈]], [[귀]]...
TRANSLATION: The skull forms the face and protects the cranial cavity. It houses the [[brain]], as well as the [[...

========== RESULT 3 ==========
ARTICLE TITLE: head
CHUNK BODY: 아주 단순한 동물의 경우 머리가 없는 것도 있으나 대부분의 [[좌우 대칭 동물류]]는 머리가 있다. [[척추동물...
TRANSLATION: Even among very simple animals, some have no head, but most [[bilaterians]] have a head. [[Vertebrat...


In [11]:
r = wiki.generate.bm25(
    query="머리말구성",
    query_properties=["chunk"],
    single_prompt="Return a translation of this into English (and nothing else): {chunk}",
    limit=3
)
for i, o in enumerate(r.objects):
    print(f"\n========== RESULT {i+1} ==========")
    print("ARTICLE TITLE:", o.properties["title"])
    print("CHUNK BODY:", o.properties["chunk"].replace("\n", " ")[:100] + "...")
    print("TRANSLATION:", o.generated.replace("\n", " ")[:100] + "...")


========== RESULT 1 ==========
ARTICLE TITLE: preface
CHUNK BODY: 적으로 머리말을 만들고 유지하는 기능을 제공하며 여기서 머리말은 페이지마다 동일할 수도 있고 페이지 번호와 같이...
TRANSLATION: It provides the ability to create and maintain headers as an adversary, where headers can be the sam...

========== RESULT 2 ==========
ARTICLE TITLE: preface
CHUNK BODY: '''머리말''' 또는 '''머리글'''은 [[타이포그래피]]에서 본문과 구별되면서도 인쇄된 페이지의 꼭대기에 ...
TRANSLATION: A 'head' or 'heading' in typography is distinct from the main text while appearing at the top of a p...

========== RESULT 3 ==========
ARTICLE TITLE: preface
CHUNK BODY: [[출판]]에서 머리말은 난외표제(running head)로도 불린다. 책에서의 일반적인 난외표제는 펼친 책의 ...
TRANSLATION: In publishing, a frontispiece is also called a running head. A common running head in a book is a ti...


In [18]:
query = "머리말구성"
r = wiki.generate.bm25(
    query=query,
    query_properties=["chunk"],
    grouped_task=f"Summarise the findings here into a few bullet points about {query}. Each point should be a single sentence, and in Korean AND English.",
    limit=10
)
print("GENERATED SUMMARY:")
print(r.generated)
for i, o in enumerate(r.objects):
    print(f"\n========== RESULT {i+1} ==========")
    print("ARTICLE TITLE:", o.properties["title"])
    print("CHUNK BODY:", o.properties["chunk"].replace("\n", " ")[:100] + "...")

GENERATED SUMMARY:
- 머리말(Preface)은 타이포그래피에서 본문과 구분되면서도 인쇄된 페이지의 꼭대기에 위치하는 텍스트입니다. - Preface or '머리글' in Korean, is text that sits at the top of a printed page, distinct from the body text in typography.
- 출판물에서 머리말은 난외표제(running head)라고도 불리며, 페이지 상단에 표시됩니다. - In publishing, prefaces are also called running heads, appearing at the top of pages.
- 워드 처리 프로그램은 일반적으로 머리말을 만들고 관리하는 기능을 제공합니다. - Word processing programs typically offer tools to create and manage prefaces.
- 머리말은 페이지마다 동일하거나 페이지 번호와 같이 달라질 수 있습니다. - Prefaces can remain the same or vary from page to page, such as with page numbers.

========== RESULT 1 ==========
ARTICLE TITLE: preface
CHUNK BODY: 적으로 머리말을 만들고 유지하는 기능을 제공하며 여기서 머리말은 페이지마다 동일할 수도 있고 페이지 번호와 같이...

========== RESULT 2 ==========
ARTICLE TITLE: preface
CHUNK BODY: '''머리말''' 또는 '''머리글'''은 [[타이포그래피]]에서 본문과 구별되면서도 인쇄된 페이지의 꼭대기에 ...

========== RESULT 3 ==========
ARTICLE TITLE: preface
CHUNK BODY: [[출판]]에서 머리말은 난외표제(running head)로도 불린다. 책에서의 일반적인 난외표제는 펼친 책의 ...

==

In [19]:
query = "머리구성"
r = wiki.generate.bm25(
    query=query,
    query_properties=["chunk"],
    grouped_task=f"Summarise the findings here into a few bullet points about {query}. Each point should be a single sentence, and in Korean AND English.",
    limit=10
)
print("GENERATED SUMMARY:")
print(r.generated)
for i, o in enumerate(r.objects):
    print(f"\n========== RESULT {i+1} ==========")
    print("ARTICLE TITLE:", o.properties["title"])
    print("CHUNK BODY:", o.properties["chunk"].replace("\n", " ")[:100] + "...")

GENERATED SUMMARY:
- 머리(Head)는 동물의 목 위에 위치하며, 머리뼈, 뇌, 눈, 귀를 포함합니다. - The head, located above the neck in animals, comprises the skull, brain, eyes, and ears.
- 머리뼈(Skull)는 얼굴을 구성하며, 뇌를 포함한 머리뼈공간을 보호합니다. - The skull forms the face and safeguards the cranial cavity, housing the brain.
- 척추동물의 배아 시기부터 신경관의 발달과 함께 머리가 형성됩니다. - Embryonic development in vertebrates marks the formation of the head alongside the growth of the neural tube.

========== RESULT 1 ==========
ARTICLE TITLE: skull
CHUNK BODY: == 구조 == [[파일:Lateral head skull.jpg|섬네일|왼쪽|머리의 구성]] 머리뼈는 얼굴을 ...

========== RESULT 2 ==========
ARTICLE TITLE: skull
CHUNK BODY: ]] 머리뼈는 얼굴을 구성하고 머리뼈공간을 보호한다. [[뇌]]를 비롯하여 [[눈 (해부학)|눈]], [[귀]]...

========== RESULT 3 ==========
ARTICLE TITLE: head
CHUNK BODY: 아주 단순한 동물의 경우 머리가 없는 것도 있으나 대부분의 [[좌우 대칭 동물류]]는 머리가 있다. [[척추동물...

========== RESULT 4 ==========
ARTICLE TITLE: literature
CHUNK BODY: 문학은 오직 문자로 이루어진 텍스트로 구성된 것만을 포함한다. 좀 더 보수적인 사람들은 그 개념이 꼭 물리적인 ...

========== RESULT 5 ==========
ARTIC